# Tensorflow and TensorRT to Frozen model

## Imports

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
from tensorflow.python.platform import gfile

from matplotlib import pyplot as plt
%matplotlib inline
from PIL import Image as PILImage

# For import utils.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

from utils import label_map_util
from utils import visualization_utils as vis_util

import pprint
import time
import cv2

### For Jetson TX2

In [3]:
from keras import backend as K

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Using TensorFlow backend.


## Variables of models
All models are in ./models folder. `Choose only one model`

In [4]:
sub_dir = time.strftime("%Y_%m_%d-%H_%M_%S", time.localtime())
print(sub_dir)

2019_07_31-14_05_28


### 1.1) ssd_mobilenet_v1_coco_2017_11_17

In [5]:
# What model to download.
MODEL_NAME = './models/ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
OUTPUT_NODES = ['num_detections', 'detection_boxes', 'detection_scores','detection_classes']

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
print(PATH_TO_LABELS)

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
print(PATH_TO_FROZEN_GRAPH)

data/mscoco_label_map.pbtxt
./models/ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb


### 1.2) faster_rcnn_resnet101_kitti_2018_01_28

In [ ]:
# What model to download.
MODEL_NAME = './models/faster_rcnn_resnet101_kitti_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
OUTPUT_NODES = ['num_detections', 'detection_boxes', 'detection_scores','detection_classes']

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'kitti_label_map.pbtxt')
print(PATH_TO_LABELS)

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
print(PATH_TO_FROZEN_GRAPH)

### 1.3) aadc2018_frcnn_res101_200k_kitti

In [ ]:
# What model to download.
MODEL_NAME = './models/aadc2018_frcnn_res101_200k_kitti'
OUTPUT_NODES = ['num_detections', 'detection_boxes', 'detection_scores','detection_classes', 'detection_masks']

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = MODEL_NAME + "/aadc_labels_2018_without_middlelane.pbtxt"
print(PATH_TO_LABELS)

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/aadc2018_frcnn_res101_200k_kitti.pb'
print(PATH_TO_FROZEN_GRAPH)

## Download Model
can download only 1.1) and 1.2) model. 

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [ ]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper Function

#### 1) Function to read ".pb" model (TensorRT model is stored in ".pb")

In [7]:
# (can be used to read frozen model or TensorRT model)
def read_pb_graph(model):
    with gfile.FastGFile(model,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def

#### 2) Function to load image into numpy array

In [8]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

#### 3) Function of showing graph

In [9]:
from IPython.display import clear_output, Image, display, HTML

# Helper functions for TF Graph visualization
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, 'utf-8')
    return strip_def
  
def show_graph(graph_def=None, max_const_size=32):
    """Visualize TensorFlow graph."""
    
    # If no input graph is given, get the default graph
    if graph_def is None:
        graph_def = tf.get_default_graph().as_graph_def()
        
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

#### 4) Showing image

In [10]:
def imshow(img):
    import cv2
    import IPython
    _,ret = cv2.imencode('.jpg',img)
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

## Images variables
There are 2 images sources. Select one of them

### 1) demo images

In [20]:
PATH_TO_TEST_IMAGES_DIR = './test_images2'
TEST_IMAGES = [ {"path": os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)), "annotations": [], "filename":'image{}.jpg'.format(i)} for i in range(1, 3) ]
pprint.pprint(TEST_IMAGES)

[{'annotations': [],
  'filename': 'image1.jpg',
  'path': './test_images2/image1.jpg'},
 {'annotations': [],
  'filename': 'image2.jpg',
  'path': './test_images2/image2.jpg'}]


### 2) Images for aadc2018_frcnn_res101_200k_kitti

In [11]:
PATH_TO_TEST_IMAGES_DIR = './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027'
TEST_IMAGE_FILENAME = [ file for file in os.listdir(PATH_TO_TEST_IMAGES_DIR) if file.endswith(('jpg', 'png','jpeg'))  ]
print(len(TEST_IMAGE_FILENAME))

FINAL_CONFIG_FILE = PATH_TO_TEST_IMAGES_DIR + "/final_config.json"
# print(FINAL_CONFIG_FILE)

import json

with open(FINAL_CONFIG_FILE) as json_file:
    final_config = json.load(json_file)
    print(len(final_config))

TEST_IMAGES = []
for filename in TEST_IMAGE_FILENAME:
    
    TEST_IMAGES.append({
        "filename" : filename,
        "path" : os.path.join(PATH_TO_TEST_IMAGES_DIR, filename),
        "annotations" : [image['annotations'] for image in final_config if image['filename']==filename][0]
    })
    
pprint.pprint(TEST_IMAGES[0])

925
925
{'annotations': [{'class': 'middle_lane',
                  'height': 32.0,
                  'width': 44.0,
                  'x': 446.0,
                  'y': 237.0},
                 {'class': 'stop_line_ahead',
                  'height': 19.0,
                  'width': 99.0,
                  'x': 483.0,
                  'y': 219.0},
                 {'class': 'child',
                  'height': 61.0,
                  'width': 70.0,
                  'x': 151.0,
                  'y': 230.0}],
 'filename': '1505828509427.jpg',
 'path': './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1505828509427.jpg'}


## Tensorflow Inference (Detection)

### Load a (frozen) Tensorflow model into memory.
.pb file is in the folder named MODEL_NAME

In [13]:
tf.reset_default_graph()

In [12]:
graph = tf.Graph()
with graph.as_default():
    with tf.Session() as sess:
        print('Read frozen model')
        tf_graph = read_pb_graph(PATH_TO_FROZEN_GRAPH)
#         show_graph(graph_def=tf_graph)
        tf.import_graph_def(tf_graph, name='')
    
        # write to tensorboard (check tensorboard for each op names)
        writer = tf.summary.FileWriter('./logs/'+sub_dir)
        writer.add_graph(sess.graph)
        writer.flush()
        writer.close()
        print("\nWrite logs {} success\n".format(sub_dir))
        
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            sess, # session
            tf.get_default_graph().as_graph_def(),# graph+weight from the session
            output_node_names=OUTPUT_NODES)

Read frozen model

Write logs 2019_07_31-14_05_28 success

INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.


### Function to inference single image

In [13]:
OUTPUTS = ['num_detections', 'detection_boxes', 'detection_scores','detection_classes', 'detection_masks']

def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in OUTPUTS:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
#             print('tensor dict')
#             pprint.pprint(tensor_dict)
            
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[1], image.shape[2])
                detection_masks_reframed = tf.cast(
                tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            print("Run Inference")
            print("image size:",image.shape)
            output_dict = sess.run(tensor_dict, feed_dict={image_tensor: image}) ## Boom here
            print("Finish Inference")
            
            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.int64)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
        return output_dict

### Function to inference all image

In [14]:
def main_inferencing(graph, limit=None):  

    total_time = 0
    correct_annotations = 0

    for idx, test_image in enumerate(TEST_IMAGES):
        image_path =test_image['path']
        image = PILImage.open(image_path)
        image_np = load_image_into_numpy_array(image)
    #     print(image_np.shape)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)

        # Actual detection.
        t1 = time.time()
        print("--")
        output_dict = run_inference_for_single_image(image_np_expanded, graph)
        print("---")
        t2 = time.time()
        delta_time = t2 - t1
        total_time += delta_time

    #     print(output_dict)

        # Visualization of the results of a detection.
        image_np, box_num = vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks'),
            use_normalized_coordinates=True,
            line_thickness=8)
        # Saving image
        image_new_path = image_path.replace('test_images_20181027','detected_images')
        # for demo images
        if image_new_path == image_path:
            image_new_path = "./test_images2/%d.jpg"%idx
        im = PILImage.fromarray(image_np, mode="RGB")
        im.save(image_new_path)
#         imshow(image_np)

        # Checking numbers of annotations
        if box_num == len(test_image['annotations']):
            correct_annotations += 1
            
        test_image['detected'] = box_num
        test_image['used_time'] = delta_time
        
        print(idx+1,'- Saving images:', test_image['filename'], ",size:",image_np.shape, "used time:", delta_time, "annotations check:", box_num == len(test_image['annotations']))

#         pprint.pprint(test_image)
        
        if limit!=None and idx+1==limit:
            break
    
    return total_time, correct_annotations

### Inferencing

#### Inference 25 images

In [13]:
print(MODEL_NAME)
total_time, correct_annotations = main_inferencing(graph,limit=25)

./models/ssd_mobilenet_v1_coco_2017_11_17
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
1 - Saving images: 1505828509427.jpg ,size: (330, 1240, 3) used time: 16.606605291366577 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
2 - Saving images: frame_348.jpeg ,size: (330, 1240, 3) used time: 12.51429271697998 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
3 - Saving images: 1534329368819.jpg ,size: (330, 1240, 3) used time: 12.536242723464966 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
4 - Saving images: 1534329390645.jpg ,size: (330, 1240, 3) used time: 12.483659267425537 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
5 - Saving images: 1534329398657.jpg ,size: (330, 1240, 3) used time: 12.472704410552979 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finis

In [14]:
acc = correct_annotations / len(TEST_IMAGES) * 100
print("Total time:",total_time)
print("Correct annotations:",correct_annotations)
print("Accuracy:",acc)

Total time: 313.6254835128784
Correct annotations: 0
Accuracy: 0.0


#### Inference all images and save output in json

In [23]:
print(MODEL_NAME)
total_time, correct_annotations = main_inferencing(graph)

./models/ssd_mobilenet_v1_coco_2017_11_17
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
1 - Saving images: 1505828509427.jpg ,size: (330, 1240, 3) used time: 11.251882553100586 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
2 - Saving images: frame_348.jpeg ,size: (330, 1240, 3) used time: 11.682080268859863 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
3 - Saving images: 1534329368819.jpg ,size: (330, 1240, 3) used time: 11.520791053771973 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
4 - Saving images: 1534329390645.jpg ,size: (330, 1240, 3) used time: 11.569775581359863 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
5 - Saving images: 1534329398657.jpg ,size: (330, 1240, 3) used time: 11.467567205429077 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Fini

--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
46 - Saving images: 1508416337654.jpg ,size: (330, 1240, 3) used time: 11.497857332229614 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
47 - Saving images: frame_842.jpeg ,size: (330, 1240, 3) used time: 11.47488784790039 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
48 - Saving images: 1534329372770.jpg ,size: (330, 1240, 3) used time: 11.756663799285889 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
49 - Saving images: 1508416291542.jpg ,size: (330, 1240, 3) used time: 11.527299165725708 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
50 - Saving images: 1534329396172.jpg ,size: (330, 1240, 3) used time: 11.3693106174469 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
51 - Saving images: 153

--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
91 - Saving images: 1506436787578.jpg ,size: (330, 1240, 3) used time: 11.555961847305298 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
92 - Saving images: 1506436788321.jpg ,size: (330, 1240, 3) used time: 11.622478723526001 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
93 - Saving images: 1505828509254.jpg ,size: (330, 1240, 3) used time: 11.593060731887817 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
94 - Saving images: 1505828199280.jpg ,size: (330, 1240, 3) used time: 11.628626585006714 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
95 - Saving images: 1534329366753.jpg ,size: (330, 1240, 3) used time: 11.616762638092041 annotations check: True
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
96 - Saving images

--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
136 - Saving images: 1534329368400.jpg ,size: (330, 1240, 3) used time: 11.431158781051636 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
137 - Saving images: 1534329211710.jpg ,size: (330, 1240, 3) used time: 11.462880849838257 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
138 - Saving images: 1534329366496.jpg ,size: (330, 1240, 3) used time: 11.509925842285156 annotations check: True
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
139 - Saving images: frame_1064.jpeg ,size: (330, 1240, 3) used time: 11.56562876701355 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
140 - Saving images: 1534329204362.jpg ,size: (330, 1240, 3) used time: 11.862448692321777 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
141 - Saving ima

--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
181 - Saving images: 1507706171602_1.jpg ,size: (330, 1240, 3) used time: 11.646157264709473 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
182 - Saving images: 1534329370625.jpg ,size: (330, 1240, 3) used time: 11.583340406417847 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
183 - Saving images: 1506436709133.jpg ,size: (330, 1240, 3) used time: 11.56594967842102 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
184 - Saving images: 1534329392764.jpg ,size: (330, 1240, 3) used time: 11.168280363082886 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
185 - Saving images: frame_1387.jpeg ,size: (330, 1240, 3) used time: 11.371597290039062 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
186 - Saving 

...

--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
721 - Saving images: 1534329367176.jpg ,size: (330, 1240, 3) used time: 11.505765676498413 annotations check: True
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
722 - Saving images: 1506437000607.jpg ,size: (330, 1240, 3) used time: 11.65730595588684 annotations check: True
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
723 - Saving images: 1534329412417.jpg ,size: (330, 1240, 3) used time: 11.502013683319092 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
724 - Saving images: 1534329204685.jpg ,size: (330, 1240, 3) used time: 11.516278505325317 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
725 - Saving images: 1534329366844.jpg ,size: (330, 1240, 3) used time: 11.465002536773682 annotations check: True
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
726 - Saving ima

--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
766 - Saving images: 1534329394918.jpg ,size: (330, 1240, 3) used time: 11.283310651779175 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
767 - Saving images: frame_258.jpeg ,size: (330, 1240, 3) used time: 11.871281623840332 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
768 - Saving images: 1534329392695.jpg ,size: (330, 1240, 3) used time: 11.54239821434021 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
769 - Saving images: 1534329373290.jpg ,size: (330, 1240, 3) used time: 11.61756682395935 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
770 - Saving images: 1534329407350.jpg ,size: (330, 1240, 3) used time: 11.4635591506958 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
771 - Saving images

--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
811 - Saving images: 1534329410618.jpg ,size: (330, 1240, 3) used time: 11.438627004623413 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
812 - Saving images: frame_1184.jpeg ,size: (330, 1240, 3) used time: 11.462392568588257 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
813 - Saving images: 1507706091675_1.jpg ,size: (330, 1240, 3) used time: 11.623680591583252 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
814 - Saving images: frame_839.jpeg ,size: (330, 1240, 3) used time: 11.65953540802002 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
815 - Saving images: 1534329398153.jpg ,size: (330, 1240, 3) used time: 11.689265727996826 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
816 - Saving ima

--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
856 - Saving images: 1534329399340.jpg ,size: (330, 1240, 3) used time: 11.311717510223389 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
857 - Saving images: 1534329174243.jpg ,size: (330, 1240, 3) used time: 11.745784521102905 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
858 - Saving images: 1534329389755.jpg ,size: (330, 1240, 3) used time: 11.721365690231323 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
859 - Saving images: 1534329404731.jpg ,size: (330, 1240, 3) used time: 10.94137454032898 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
860 - Saving images: 1534329404155.jpg ,size: (330, 1240, 3) used time: 11.147644519805908 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
861 - Saving 

--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
901 - Saving images: 1534329402762.jpg ,size: (330, 1240, 3) used time: 11.768301963806152 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
902 - Saving images: 1534329370467.jpg ,size: (330, 1240, 3) used time: 11.132109642028809 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
903 - Saving images: 1534329212981.jpg ,size: (330, 1240, 3) used time: 11.476155281066895 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
904 - Saving images: 1534329206564.jpg ,size: (330, 1240, 3) used time: 11.728055953979492 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
905 - Saving images: frame_170.jpeg ,size: (330, 1240, 3) used time: 11.43232774734497 annotations check: False
--
Run Inference
image size: (1, 330, 1240, 3)
Finish Inference
---
906 - Saving ima

In [24]:
# writing json file
JSON_TO_SAVE = './models/aadc2018_frcnn_res101_200k_kitti/detected_images/images_detail.json'

with open(JSON_TO_SAVE, "w") as write_file:
    json.dump(TEST_IMAGES, write_file , indent = 4, sort_keys=True )

In [25]:
acc = correct_annotations / len(TEST_IMAGES) * 100
print("Total time:",total_time)
print("Correct annotations:",correct_annotations)
print("Accuracy:",acc)

Total time: 10683.43048787117
Correct annotations: 48
Accuracy: 5.189189189189189


### Show deteced images
for images of aadc2018_frcnn_res101_200k_kitti. If for demo images path is './test_images2'

In [ ]:
PATH_TO_DETECTED_IMAGES_DIR = './test_images2'
# PATH_TO_DETECTED_IMAGES_DIR = './models/aadc2018_frcnn_res101_200k_kitti/detected_images/'
DETECTED_IMAGE_PATHS = [ os.path.join(PATH_TO_DETECTED_IMAGES_DIR, file) for file in os.listdir(PATH_TO_DETECTED_IMAGES_DIR) if file.endswith("jpg") ]
print(DETECTED_IMAGE_PATHS)

In [ ]:
IMAGE_SIZE = (12, 8)
for idx, image_path in enumerate(DETECTED_IMAGE_PATHS):
    image = PILImage.open(image_path)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(np.asarray(image))

In [ ]:
# Or...showing in table
IMAGE_SIZE = (12, 8)
plt.figure(figsize=IMAGE_SIZE)
COLS = 1
ROWS = len(DETECTED_IMAGE_PATHS) / COLS+1

for idx, image_path in enumerate(DETECTED_IMAGE_PATHS):
    image = PILImage.open(image_path)
#     image.show()
    plt.subplot(ROWS, COLS, idx+1 )
    plt.imshow(np.asarray(image))

## Interferencing With TensorRT

### Optimize to TensorRT Model

In [ ]:
BATCH_SIZE = 1 # Decrease Batch size decrease time of optimizing and using memory while inferencing.

#### try precision mode as FP32

In [15]:
%%time
# outputs = [out.op.name for out in model.outputs] # name of output layer

# convert (optimize) frozen model to TensorRT model
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,# frozen model
    outputs=OUTPUT_NODES,
    max_batch_size=BATCH_SIZE,# specify your max batch size
    max_workspace_size_bytes=5*(10**9),# specify the max workspace
    precision_mode="FP32") # precision, can be "FP32" (32 floating point precision) or "FP16"

CPU times: user 1min, sys: 9.48 s, total: 1min 9s
Wall time: 1min 40s


In [16]:
#write the TensorRT model to be used later for inference
pb_trt_filename = "trt_frozen_graph-"+sub_dir+"_FP32.pb"
PATH_TO_TRT_FROZEN_GRAPH = MODEL_NAME+ "/" + pb_trt_filename
print(PATH_TO_TRT_FROZEN_GRAPH)

./models/ssd_mobilenet_v1_coco_2017_11_17/trt_frozen_graph-2019_07_31-14_05_28_FP32.pb


In [17]:
with gfile.FastGFile(PATH_TO_TRT_FROZEN_GRAPH, 'wb') as f:
    f.write(trt_graph.SerializeToString())
print(pb_trt_filename + " is successfully stored!")

trt_frozen_graph-2019_07_31-14_05_28_FP32.pb is successfully stored!


#### try precision mode as FP16

In [17]:
%%time
# outputs = [out.op.name for out in model.outputs] # name of output layer

# convert (optimize) frozen model to TensorRT model
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,# frozen model
    outputs=OUTPUT_NODES,
    max_batch_size=BATCH_SIZE,# specify your max batch size
    max_workspace_size_bytes=5*(10**9),# specify the max workspace
    precision_mode="FP16") # precision, can be "FP32" (32 floating point precision) or "FP16"

CPU times: user 1min 8s, sys: 37.5 s, total: 1min 45s
Wall time: 12min 3s


In [33]:
#write the TensorRT model to be used later for inference
pb_trt_filename = "trt_frozen_graph-"+sub_dir+"_FP16.pb"
PATH_TO_TRT_FROZEN_GRAPH = MODEL_NAME+ "/" + pb_trt_filename
print(PATH_TO_TRT_FROZEN_GRAPH)

./models/ssd_mobilenet_v1_coco_2017_11_17/trt_frozen_graph-2019_07_26-12_30_15_FP16.pb


In [19]:
with gfile.FastGFile(PATH_TO_TRT_FROZEN_GRAPH, 'wb') as f:
    f.write(trt_graph.SerializeToString())
print(pb_trt_filename + " is successfully stored!")

trt_frozen_graph-2019_07_26-12_30_15_FP16.pb is successfully stored!


#### try precision mode as INT8

In [20]:
%%time
# outputs = [out.op.name for out in model.outputs] # name of output layer

# convert (optimize) frozen model to TensorRT model
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,# frozen model
    outputs=OUTPUT_NODES,
    max_batch_size=BATCH_SIZE,# specify your max batch size
    max_workspace_size_bytes=5*(10**9),# specify the max workspace
    precision_mode="INT8") # precision, can be "FP32" (32 floating point precision) or "FP16"

CPU times: user 16.1 s, sys: 216 ms, total: 16.3 s
Wall time: 16.2 s


In [21]:
#write the TensorRT model to be used later for inference
pb_trt_filename = "trt_frozen_graph-"+sub_dir+"_INT8.pb"
PATH_TO_TRT_FROZEN_GRAPH = MODEL_NAME+ "/" + pb_trt_filename
print(PATH_TO_TRT_FROZEN_GRAPH)

./models/ssd_mobilenet_v1_coco_2017_11_17/trt_frozen_graph-2019_07_26-12_30_15_INT8.pb


In [22]:
with gfile.FastGFile(PATH_TO_TRT_FROZEN_GRAPH, 'wb') as f:
    f.write(trt_graph.SerializeToString())
print(pb_trt_filename + " is successfully stored!")

trt_frozen_graph-2019_07_26-12_30_15_INT8.pb is successfully stored!


### Load TensorRT graph

After trying to load TenzorRT frozen graph file (ssd_mobilenet_v1_coco_2017_11_17 .pb file) in 3 different precision mode.
1. FP32<br>
    Convert from TensotFlow to TensorRT take time around 9 minutes.
    Can load the model.
2. FP16<br>
    Convert from TensotFlow to TensorRT take time around 12 minutes.
    Can load the model.  
3. INT8<br>
    Convert from TensotFlow to TensorRT take time around 16 seconds!!.
    Cannot! load the model.

In [18]:
# Exist TensorRT frozen graph path from the same model but different procision mode. Select one of three
# PATH_TO_TRT_FROZEN_GRAPH = "./models/ssd_mobilenet_v1_coco_2017_11_17/trt_frozen_graph-2019_07_26-12_30_15_FP32.pb"
# PATH_TO_TRT_FROZEN_GRAPH = "./models/ssd_mobilenet_v1_coco_2017_11_17/trt_frozen_graph-2019_07_26-12_30_15_FP16.pb"
# PATH_TO_TRT_FROZEN_GRAPH = "./models/ssd_mobilenet_v1_coco_2017_11_17/trt_frozen_graph-2019_07_26-12_30_15_INT8.pb"
# PATH_TO_TRT_FROZEN_GRAPH = "./models/ssd_mobilenet_v1_coco_2017_11_17/trt_frozen_graph-2019_07_18-17_23_40.pb"
PATH_TO_TRT_FROZEN_GRAPH = "./models/ssd_mobilenet_v1_coco_2017_11_17/trt_frozen_graph-2019_07_31-14_05_28_FP32.pb"
print(PATH_TO_TRT_FROZEN_GRAPH)

./models/ssd_mobilenet_v1_coco_2017_11_17/trt_frozen_graph-2019_07_31-14_05_28_FP32.pb


In [19]:
tf.reset_default_graph()

In [20]:
%%time
trt_detection_graph = tf.Graph()
with trt_detection_graph.as_default():
    with tf.Session() as sess:
        print('Read TensorRT frozen model')
        trt_graph = read_pb_graph(PATH_TO_TRT_FROZEN_GRAPH)
#         show_graph(graph_def=tf_graph)
        tf.import_graph_def(trt_graph, name='')
#         print([node.name for node in trt_graph.node])
        
        # write to tensorboard (check tensorboard for each op names)
        writer = tf.summary.FileWriter('./trt_logs/'+sub_dir+'_FP32')
#         writer = tf.summary.FileWriter('./trt_logs/FP32')
        writer.add_graph(sess.graph)
        writer.flush()
        writer.close()
        print("\nWrite trt_logs success")

# print([node.name for node in trt_detection_graph.get_operations()])

Read TensorRT frozen model

Write trt_logs success
CPU times: user 15.5 s, sys: 164 ms, total: 15.7 s
Wall time: 15.7 s


### Inference

In [ ]:
trt_total_time, trt_correct_annotations = main_inferencing(trt_detection_graph, limit=25)

--
Run Inference
image size: (1, 330, 1240, 3)


In [ ]:
trt_acc = trt_correct_annotations / TEST_IMAGES * 100
print("TensorRT Total time:",trt_total_time)
print("TensorRT Correct annotations:",trt_correct_annotations)
print("TensorRT Accuracy:", trt_acc)

## Different Time

There is once time that can inference both TensorFlow and TensorRT. The inferencing time of TensorRT is faster than TensorFlow about 1 second when using `ssd_mobilenet_v1_coco_2017_11_17` model and demo images.

In [ ]:
print("Detection time: ", total_time)
print("With TensorRT Detection time: ", trt_total_time)
print("Different time (TensorRT faster):", total_time - trt_total_time)
print("TensorRT improvement:", total_time/trt_total_time, "times")